In [1]:
import xarray as xr
xr.set_options(display_style='html')
import intake
import cftime
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset
from matplotlib.colors import LogNorm
import cartopy.crs as ccrs
import cartopy

In [2]:
year = 2014
AR = xr.open_dataset(str(year)+'_crit.nc')

In [3]:
ivt_ = AR.ivt
ivt = xr.concat([ivt_,ivt_[:,:,:5]],dim='lon')# add extra points for AR at the edge of the world

In [7]:
out_ar = ivt.copy()
out_ar.values[:] = out_ar.values[:]*0.0

def floodfill(indata):
    outloc = np.copy(indata)
    for x in range(indata.shape[0]):
        ind_lim = np.where(indata[x,:]==1)
        for ii in ind_lim:
            if len(ii)>1:
                jj=0
                while jj<=len(ii)-2:
                    if abs(ii[jj]-ii[jj+1])>10:
                        jj=jj+1
                    else:
                        outloc[x,ii[jj]:ii[jj+1]]=1
                        jj=jj+1
    return outloc


for k in range(ivt.shape[0]):
    matrix = ivt[k,:,:]
    tst = floodfill(matrix)
    out_ar[k,:,:] = tst
    
# add the values of the extra points to get ARs at the end of the world
out_ar[:,:,:5] = out_ar[:,:,:5]+out_ar[:,:,144:]
out_ar = out_ar[:,:,:144]
out_ar = xr.where(out_ar>1,1,out_ar)


lat_ = AR.lat
lon_ = AR.lon

out_ar.to_netcdf(str(year)+'_AR_detection.nc')